<a href="https://colab.research.google.com/github/vunison/ds/blob/main/%D1%81%D0%B5%D0%B3%D0%BC%D0%B5%D0%BD%D1%82%D0%B0%D1%86%D0%B8%D1%8F_%D0%BA%D0%BB%D0%B8%D0%B5%D0%BD%D1%82%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Нужно распределить имеющихся клиенетов по сегментам, приблизительно:

УК  
школы  
гостиницы  
детские сады  
администрации  
базы отдыха  
строительные компании  
благотворительные фонды  
больницы  
ИП+юр.лица  
физ.лица  




In [ ]:
!pip install catboost
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import joblib
from joblib import load

import pandas as pd
from pymorphy2 import MorphAnalyzer
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


from sklearn.feature_extraction.text import TfidfVectorizer
from catboost import CatBoostClassifier

from tqdm import tqdm

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
stop_words = set(stopwords.words('russian'))

In [ ]:
model_fiz = joblib.load('model_fiz.joblib')
model_ur = joblib.load('model_ur.joblib')

tvidf_v_fiz = joblib.load('tvidf_v_fiz.joblib')
tvidf_v_ur =   joblib.load('tvidf_v_ur.joblib')

# Предобработка

In [ ]:
data = pd.read_csv('/content/s-o-t-04-08-2022.csv').drop(['Phone 3 - Value','Subscribed'],axis=1)

In [ ]:
data.columns = ['order', 'name', 'organization',
       'address', 'group membership', 'e-mail',
       'phone1', 'phone2']

In [ ]:
def fast_clean_sub(text):
  name = re.sub('[0-9*&?]','',text)
  return name
data['name'] =  data['name'].astype(str)
data['name'] =  data['name'].apply(fast_clean_sub)

data['organization'].loc[data[data['organization'].isna() == True].index] = data['name'].loc[data[data['organization'].isna() == True].index]
data['organization'].loc[data[data['organization'] == 'Название компании'].index] = data['name'].loc[data[data['organization'].isna() == True].index]

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
data = data.dropna(subset=['organization'])

In [ ]:
data['organization'] = data['organization'].astype(str)
data['organization'] = data['organization'].str.lower().str.replace('-',' ', regex=True)
data['organization'] = data['organization'].str.replace('\\','', regex=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

# Лемматизация

In [ ]:
m = MorphAnalyzer()
data['lemm_organization'] = [' '.join([m.parse(word)[0].normal_form for word in x.split()]) for x in tqdm(data['organization'])]

100%|██████████| 9315/9315 [00:13<00:00, 669.84it/s]


# Векторизация

In [ ]:
vectorized_lemmas_fiz = tvidf_v_fiz.transform(data['lemm_organization'])

# Присвоение категорий - ФИЗ/ЮР

In [ ]:
data['category_1'] = model_fiz.predict(vectorized_lemmas_fiz)

In [ ]:
other = ['тендер','некрасов','кп','тест','клиент','неизвестно','nan','шк','шемякина','dna','шемякин']

other_idx = []
for idx in data.index:
  for word in other:
    if word in (data['lemm_organization'][idx]).split():
      other_idx.append(idx)
data['category_1'][other_idx] = 'другое'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# Присвоение категорий - ЮР ЛИЦ

In [ ]:
data_ur = data[data['category_1'] == 'юр лица']
vectorized_lemmas_ur = tvidf_v_ur.transform(data_ur['lemm_organization'])
data_ur['category_2'] = model_ur.predict(vectorized_lemmas_ur)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
data_full = data.join(data_ur[['category_2']], how='left').reset_index(drop=True)

## Чистим телефон от лишнего

In [ ]:
data_full['phone1'] = data_full['phone1'].astype(str) 

In [ ]:
def clean_phone(phone):
  return re.sub('[+() -]','',phone)
data_full['phone1'] = data_full['phone1'].apply(clean_phone)

Процент дубликатов в телефоне

In [ ]:
data_full['phone1'].duplicated().sum()/len(data_full)

0.27471819645732687

In [ ]:
data_full = data_full[data_full['category_1']!='другое']

In [ ]:
data_full['phone1'].value_counts().head(10)

70000000000    901
79827907667     40
74164350956     14
79143905731     12
71111111111     11
79999999999     10
79118700597     10
74739653792     10
77017446973      9
79214885258      9
Name: phone1, dtype: int64